In [1]:
import mouse
import keyboard
import time
from PIL import Image,ImageGrab
from math import sqrt
from mouse import LEFT, DOWN, get_position, wait
from myani2sketch import ani2sketch
import pickle
import random
import colorsys

MAINDIR ="./"
SLEEPCLICK, SLEEPMOVE = 0.01,0.00000001
stopDrawing = False
linemode = True
# OrderMode = "row", "column", "spiral", "random", "nna"

rgbRange=[0,64,128,192,255]
rgb2index = {} 

class Color:
    def __init__(self,name,r,g,b,mx,my,mz=-1):
        self.name = name
        self.R = r  
        self.G = g  
        self.B = b  
        self.RGB = (self.R,self.G,self.B)
        self.x = int(mx)  # also 1.0 index
        self.y = int(my)  # also 0.5 index
        self.z = int(mz)  # 0.5 white flag
        self.xy = (self.x,self.y)  
    def printData(self):
        print(f"{self.name}: {self.R},{self.G},{self.B}, X: {self.x} Y: {self.y}")

def GetPosition(positionname):
    print(positionname)
    mouse.wait(LEFT, mouse.DOWN)
    posi = get_position()
    return posi

def Exit():
    global stopDrawing
    stopDrawing = True
    print("Exiting")
    
def clickPen(posi):
    mouse.move(posi[0], posi[1], duration=0)
    time.sleep(SLEEPMOVE)
    mouse.click(LEFT)
    time.sleep(SLEEPCLICK) 

def getImageAndCanvas(relocate,testimg):
    if testimg:
        imgfile = Image.open(MAINDIR+"testimg/"+str(testimg)+".jpg")
    else:
        imgfile = ImageGrab.grabclipboard()
        if type(imgfile) is type(None):
            print("No Image.")
            return None,(0,0)
    if relocate:
        x0y0,xnyn = GetPosition("x0y0"),GetPosition("xnyn")
    else: 
        x0y0,xnyn = clefttop,crightbottom
        
    img = imgfile.convert("RGB")
    canvas = (abs(xnyn[0]-x0y0[0]),abs(xnyn[1]-x0y0[1]))
    xSize, ySize = img.size
    ratio = min(canvas[0]/xSize, canvas[1]/ySize)
    img = img.resize((int(xSize*ratio),int(ySize*ratio)))
    
    return img,(min(xnyn[0],x0y0[0]),min(xnyn[1],x0y0[1]))

In [20]:
def nnaSearchList(x,y):
    return [(x,y+1),(x+1,y+1),(x+1,y),(x+1,y-1),(x,y-1),(x-1,y-1),(x-1,y),(x-1,y+1)]
def nna(colorXYs,x,y,xsize,ysize):
    for (nextx,nexty) in nnaSearchList(x,y):
        if nextx in range(xsize) and nexty in range(ysize):
            if colorXYs[nextx][nexty]:
                for (a,b) in nnaSearchList(x,y):
                    if a in range(xsize) and b in range(ysize):
                        colorXYs[a][b] = 0
                colorXYs[nextx][nexty] = 0
                return [(nextx,nexty)] + nna(colorXYs,nextx,nexty,xsize,ysize)
    return []
    
def drawOrder(colorXYs, ordermode):
    order=[]
    xsize = len(colorXYs)
    ysize = len(colorXYs[0])
    if ordermode == "column":
        for x in range(xsize):
            for y in range(ysize):
                if colorXYs[x][y]: order.append((x,y))
    if ordermode == "row":
        for y in range(ysize):
            for x in range(xsize):
                if colorXYs[x][y]: order.append((x,y))
    if ordermode == "random":
        for x in range(xsize):
            for y in range(ysize):
                if colorXYs[x][y]: order.append((x,y))
        random.shuffle(order)
    if ordermode == "spiral":
        m,n = xsize,ysize
        k = 0
        l = 0
        while (k < m and l < n):
            for i in range(l, n):
                if colorXYs[k][i]: order.append((k,i))
            k += 1
            for i in range(k, m):
                if colorXYs[i][n - 1]: order.append((i,n-1))
            n -= 1
            if (k < m):
                for i in range(n - 1, (l - 1), -1):
                    if colorXYs[m - 1][i]: order.append((m-1,i))
                m -= 1
            if (l < n):
                for i in range(m - 1, k - 1, -1):
                    if colorXYs[i][l]: order.append((i,l))
                l += 1
        order.reverse()
    if ordermode == "nna":
        nnaLists1 = []
        nnaLists2 = []
        nnaLists3 = []
        for y in range(ysize):
            for x in range(xsize):
                if colorXYs[x][y]: 
                    colorXYs[x][y] = 0
                    nnaResult = [(x,y)]+nna(colorXYs,x,y,xsize,ysize)
                    if len(nnaResult)> 25:
                        nnaLists1.append(nnaResult)
                    elif len(nnaResult)> 5:
                        nnaLists2.append(nnaResult)
                    else:
                        nnaLists3.append(nnaResult)
        for nnaResult in nnaLists1:
            order += nnaResult
        for nnaResult in nnaLists2:
            order += nnaResult
        for nnaResult in nnaLists3:
            order += nnaResult
    return order

In [5]:
def getColorPanel(img,maxy,maxx,pLeftTop,pRightBottom):
    xsize,ysize = abs(pLeftTop[0]-pRightBottom[0])-4,abs(pRightBottom[1]-pLeftTop[1])-4
    x0,y0 = pLeftTop[0]+2,pLeftTop[1]+2
    
    panelColors = []
    blackIndex,whiteIndex = 0,124
    i =0 
    for r in rgbRange:
        for g in rgbRange:
            for b in rgbRange:
                rgb = (r,g,b)
                rgb2index[rgb]=i
                xScale,yScale = rgb2xy(rgb)
                x = int(xsize * xScale) + x0
                y = int(ysize * yScale) + y0
                panelColors.append(Color("c"+str(i),r,g,b,x,y))
                if r==255 and g==255 and b==255: whiteIndex = i
                if r==0   and g==0   and b==0:   blackIndex = i
                i += 1
    return panelColors,blackIndex,whiteIndex

def getClosest(val1, val2, target):
    if (target - val1 >= val2 - target): return val2
    else:  return val1
def findClosest(arr, target):
    n = len(arr)
    if (target <= arr[0]):return arr[0]
    if (target >= arr[n - 1]):return arr[n - 1]
    i = 0; j = n; mid = 0
    while (i < j):
        mid = (i + j) // 2
        if (arr[mid] == target): return arr[mid]
        if (target < arr[mid]) :
            if (mid > 0 and target > arr[mid - 1]):
                return getClosest(arr[mid - 1], arr[mid], target)
            j = mid
        else :
            if (mid < n - 1 and target < arr[mid + 1]):
                return getClosest(arr[mid], arr[mid + 1], target)
            i = mid + 1
    return arr[mid]

def FindClosestColor(rgb: tuple):
    r,g,b = rgb
    closestR = findClosest(rgbRange,r)
    closestG = findClosest(rgbRange,g)
    closestB = findClosest(rgbRange,b)
    closestColor = (closestR,closestG,closestB)
    return rgb2index[closestColor]

In [4]:
def rgb2xy(rgb):
    r,g,b = rgb
    r, g, b = r/255.0,g/255.0,b/255.0
    h, l, s = colorsys.rgb_to_hls(r, g, b) #s=1
    return h,1-l
    
def getPixelSets(img,rgbDetail,skeDetail):  
    xSize, ySize = img.size        
    pixRGB = img.load()
    pixSKT = ani2sketch(img).load() 
    rgbcolorsXYs = [[[0 for y in range(ySize)] for x in range(xSize)] for i in panelColors]
    sketchColorsXYs = [[[0 for y in range(ySize)] for x in range(xSize)]]
    
    for y in range(ySize): 
        for x in range(xSize):
            #if rgbDetail and not x%rgbDetail and not y%rgbDetail:  
            if rgbDetail:
                r,g,b = pixRGB[x, y]
                Index1 = FindClosestColor((r,g,b))
                if Index1 != whiteIndex:
                    rgbcolorsXYs[Index1][x][y]=1
            #if skeDetail and not x%skeDetail and not y%skeDetail:
            if skeDetail:
                r,g,b = pixSKT[x, y]
                if r<240 or g<240 or b<240: #skip white
                    sketchColorsXYs[0][x][y]=1
    return  rgbcolorsXYs,sketchColorsXYs

In [8]:
def drawSingleColor(isColorXYs,x0y0,step,ordermode):
    global linemode,stopDrawing
    colorXYs = drawOrder(isColorXYs, ordermode)
    if linemode:
        lastx,lasty,ishold = -1,-1,0
        for (x,y) in colorXYs:
            if(stopDrawing): 
                mouse.release()
                return True
            if x%step or y%step: continue  # comment this if in getPixelSets already not x%rgbDetail and not y%rgbDetail
            if abs(x-lastx) <= (step) and abs(y-lasty) <=(step): 
                mouse.hold()
            else: 
                time.sleep(SLEEPCLICK)
                mouse.release()
                ishold=0
            mouse.move(x0y0[0]+x,x0y0[1]+y, duration=0)
            time.sleep(SLEEPMOVE)
            if not ishold:
                mouse.click()
                ishold = 1
            lastx,lasty = x, y
        time.sleep(SLEEPCLICK)
        mouse.release()
    else:
        for (x,y) in colorXYs:
            if(stopDrawing): return True
            clickPen((x0y0[0]+step*x,x0y0[1]+step*y))
    return False

def drawColor(testimg,relocate,rgbInfo,skeInfo):
    global stopDrawing    
    rgbDetail,rgbOrder = rgbInfo
    skeDetail,skeOrder = skeInfo
    img,x0y0 = getImageAndCanvas(relocate,testimg)
    if not img: return
    
    rgbPixelSet,skePixelSet = getPixelSets(img,rgbDetail,skeDetail)
    rgbcolorsXYs = rgbPixelSet
    sketchColorsXYs = skePixelSet
    
    if rgbDetail:
        stopDrawing = False
        for isColorXYs in rgbcolorsXYs:
            if(stopDrawing): continue
            time.sleep(0.01)
            clickPen(colorpicker)
            time.sleep(0.01)
            clickPen(panelColors[rgbcolorsXYs.index(isColorXYs)].xy)
            stopDrawing = drawSingleColor(isColorXYs,x0y0,rgbDetail,rgbOrder)
            
    if skeDetail:
        stopDrawing = False
        for isColorXYs in sketchColorsXYs:
            if(stopDrawing): continue
            time.sleep(0.01)
            clickPen(colorpicker)
            time.sleep(0.01)
            clickPen(panelColors[blackIndex].xy)
            stopDrawing = drawSingleColor(isColorXYs,x0y0,skeDetail,skeOrder)

In [6]:
keyboard.add_hotkey("esc", Exit)
colorpicker = GetPosition("colorpicker")
pen1 = GetPosition("pen1")
pLeftTop = GetPosition("mLeftTop")
pRightBottom = GetPosition("mRightBottom")

colorpicker
pen1
mLeftTop
mRightBottom


In [18]:
clefttop= GetPosition("clefttop")
crightbottom = GetPosition("crightbottom")

clefttop
crightbottom


In [13]:
panelColors,blackIndex,whiteIndex = getColorPanel(None,7,12,pLeftTop,pRightBottom)

In [17]:
drawColor(testimg=6,relocate=False,rgbInfo=(1,"row"),skeInfo=(0,"row"))

In [21]:
drawColor(testimg=6,relocate=False,rgbInfo=(0,"row"),skeInfo=(1,"nna"))